## Heralded Coincidence Time trace

For this section, we would like to produce a Coinciden Time Trace Plot of our heralded photon experiment.
The required imports are as followed:

In [1]:
from snAPI.Main import *
import matplotlib
matplotlib.use('TkAgg',force=True)
from matplotlib import pyplot as plt
print("Switched to:",matplotlib.get_backend())

Switched to: TkAgg


The usage of snAPI modules are Object Oriented Based, which meant that you would have to initialise an object class.

In [2]:
sn = snAPI()

Since we are analysizing a PTU file instead of spontaneously obtaining measurements from the TimeHarp 260 Nano, we only need sn.getFileDevice(). If we were spontaneously obtaining measurements on the fly, we would have required sn.getDevice().

We also need to set the measuring mode. we have ours set to MeasMode.T2, which basically meant that out measurements only took 2 channels, channel 1 and channel 2.(T3 would include an extra sync channel)

There is also a need to setup the sn object to include the configuration for TH260N.

In [3]:
sn.getFileDevice(r"antibunch.sptw\down_con_herald2.ptu")
sn.setLogLevel(LogLevel.DataFile, True) # might be optional, it looks like a debug function
sn.initDevice(MeasMode.T2)
sn.loadIniConfig("config\TH260N.ini")

<>:4: SyntaxWarning: invalid escape sequence '\T'
<>:4: SyntaxWarning: invalid escape sequence '\T'
C:\Users\andre\AppData\Local\Temp\ipykernel_18812\2789257668.py:4: SyntaxWarning: invalid escape sequence '\T'
  sn.loadIniConfig("config\TH260N.ini")


0

The remaining code pertains to setting the configuration of the plot.

In [4]:


# what is window time
coincidenceAll = sn.manipulators.coincidence([1,2], windowTime=1e7, mode=CoincidenceMode.CountAll, keepChannels=True)
coincidenceOnce = sn.manipulators.coincidence([1,2], windowTime=1e7, mode=CoincidenceMode.CountOnce, keepChannels=True)

# the number of bins we need is rougthly 1000s (round up from 714s) divided by the average period of coincidence which is 1ms,
# we should expect to see around 1 coincidence or so on the countrate axis, but instead it's a few orders more
# expecting 1 coincidence per millisecond
sn.timeTrace.setNumBins(10000) # 1000s/10ms = 10_000

# Our acqusition time was 714 seconds, so we plot for 720s
sn.timeTrace.setHistorySize(720) # 720s

# 10000 is ignore when we use a PTU file instead, only relevant when connected to device
sn.timeTrace.measure(10000, waitFinished=True, savePTU=False)

counts, times = sn.timeTrace.getData()
plt.clf()

#what's the difference between coincidence all and coincidence once
plt.plot(times, counts[coincidenceAll], linewidth=2.0, label='coincidenceAll')
plt.plot(times, counts[coincidenceOnce], linewidth=2.0, label='coincidenceOnce')

plt.xlabel('Time [s]')
plt.ylabel('Counts[Cts/10ms]')
plt.legend()
plt.title("TimeTrace")
plt.show()

Below is a HeraldedCorrelationG2 

In [5]:
# from snAPI.Main import *
# import matplotlib
# matplotlib.use('TkAgg',force=True)
# from matplotlib import pyplot as plt
# print("Switched to:",matplotlib.get_backend())
# import time

# if(__name__ == "__main__"):

#     sn = snAPI()
    
#     sn.getFileDevice(r"antibunch.sptw\down_con_herald2.ptu")
#     sn.initDevice(MeasMode.T2)
#     sn.loadIniConfig("config\TH260N.ini")
    
#     #windowSize = 2000 #ps
#     windowSize = 2000
#     #windowSize = 2000000000000 #ps

#     # move the histograms of the both channels to the same time after the sync to filter both at once
#     sn.device.setInputChannelOffset(0,-4000)

#     # measure the count rate before the herald filter
#     CrInIdx = sn.manipulators.countrate()
    
#     # define a gate window after the herald channel 0 (sync) for the detector channels 1 and 2, 
#     # starting at 52000 ps after the herald signal , with a gate length of windowSize (300 ps)
#     heraldChans = sn.manipulators.herald(0, [1,2], 46000, windowSize, inverted=False, keepChannels=True )
    
#     # measure the count rate after the herald filter
#     CrOutIdx = sn.manipulators.countrate()

#     # initiate the g2 correlation with the time-gated channels
#     sn.correlation.setG2Parameters(heraldChans[0], heraldChans[1], windowSize, 1)
#     sn.correlation.measure(10000,savePTU=False)

#     while True:
#         finished = sn.correlation.isFinished()
#         data, bins = sn.correlation.getG2Data()
        
#         CRin = sn.manipulators.getCountrates(CrInIdx)
#         CRout = sn.manipulators.getCountrates(CrOutIdx)
        
#         # print the count rates
#         sn.logPrint(f"CR in: {CRin[1]}, {CRin[2]} - out: {CRout[heraldChans[0]]}, {CRout[heraldChans[1]]}")
        
#         time.sleep(.3)
        
#         plt.clf()
#         plt.plot(bins, data, linewidth=2.0, label='g(2)')
#         plt.xlabel('Time [s]')
#         plt.ylabel('g(2)')
#         plt.legend()
#         plt.title("g(2)")
#         plt.pause(0.1)
        
#         if finished:
#             break
    
#     plt.show(block=True)